In [1]:
import hashlib
import pandas as pd
import numpy as np

# データの生成

## 作成カラム

- **会員番号**: 5桁の文字型
- **契約日**: 2020年1月1日から2024年12月31日の間でランダムな日付（yyyymmdd形式）
- **性別**: 男性と女性がそれぞれ60%と40%の割合
- **契約プラン**: ベーシック、ライト、プレミアのプランがそれぞれ5:3:2の割合

In [8]:
# シード値とパラメータを設定
np.random.seed(0)
num_samples = 500

# 会員番号5桁の文字型
member_ids = np.random.choice(range(1, 1001),
                              size=num_samples,
                              replace=False)
# 契約日 2020年1月1日から2024年12月31日の間でランダムな日付（yyyymmdd）
contract_dates = pd.to_datetime(np.random.choice(pd.date_range(start='2020-01-01',
                                                               end='2024-12-31'),
                                                 size=num_samples))
# 性別 男性と女性がそれぞれ60%と40%の割合
genders = np.random.choice(['男性', '女性'],
                           size=num_samples,
                           p=[0.6, 0.4])
# 契約プラン　ベーシック、ライト、プレミアのプランが5:3:2の割合
plans = np.random.choice(['ベーシック', 'ライト', 'プレミア'],
                         size=num_samples,
                         p=[0.5, 0.3, 0.2])

# DataFrameの作成
df_members = pd.DataFrame({
    '会員番号': np.array([f'{i:05d}' for i in member_ids]),
    '契約日': contract_dates.strftime('%Y%m%d'),
    '性別': genders,
    '契約プラン': plans
})

In [9]:
df_members.head()

,会員番号,契約日,性別,契約プラン
0,00994,20230828,女性,ベーシック
1,00860,20240612,男性,プレミア
2,00299,20210522,女性,ベーシック
3,00554,20230630,男性,ベーシック
4,00673,20210519,男性,ライト


# データの確認
### 指定の要件を満たしていることを確認する

#### データ確認における`assert`文の使用

`assert`文は、特定の条件が真であることを確認するために使用される。データの整合性や前提条件を検証する際に特に有用。

##### 使用方法

- `assert 条件, エラーメッセージ`
  - 条件: 確認したい条件式
  - エラーメッセージ: 条件がFalseの場合に表示されるメッセージ

##### 例

以下のコードは、データフレーム内の`gender`列が正しく設定されているかを確認する例です。

```python
assert df['gender'].isin(['男性', '女性']).all(), "性別は男性または女性でなければなりません"


In [10]:
try:
    # 会員番号の検証: 500件の会員番号がユニークである
    assert df_members['会員番号'].nunique() == 500, "会員番号が500件のユニークな値ではありません。"

    # 契約日の検証: 2020年1月1日から2024年12月31日の間
    min_date, max_date = pd.to_datetime("2020-01-01"), pd.to_datetime("2024-12-31")
    assert df_members['契約日'].apply(lambda x: min_date <= pd.to_datetime(x, format='%Y%m%d') <= max_date).all(), "契約日が指定の範囲外です。"

    # 性別の割合検証: 男性6:女性4
    gender_ratio = df_members['性別'].value_counts(normalize=True)
    assert abs(gender_ratio['男性'] - 0.6) < 0.05, "男性の割合が60%から大きく外れています。"
    assert abs(gender_ratio['女性'] - 0.4) < 0.05, "女性の割合が40%から大きく外れています。"

    # 契約プランの割合検証: ベーシック、ライト、プレミアのプランが5:3:2の割合
    plan_ratio = df_members['契約プラン'].value_counts(normalize=True)
    assert abs(plan_ratio['ベーシック'] - 0.5) < 0.05, "ベーシックプランの割合が50%から大きく外れています。"
    assert abs(plan_ratio['ライト'] - 0.3) < 0.05, "ライトプランの割合が30%から大きく外れています。"
    assert abs(plan_ratio['プレミア'] - 0.2) < 0.05, "プレミアプランの割合が20%から大きく外れています。"

    "データは指定の要件で作成されています。"
except AssertionError as e:
    str(e)

In [11]:
print('会員番号ユニーク数', df_members['会員番号'].nunique())
print('契約日min', df_members['契約日'].min())
print('契約日max', df_members['契約日'].max())
display(df_members['性別'].value_counts(normalize=True))
display(df_members['契約プラン'].value_counts(normalize=True))

会員番号ユニーク数 500
契約日min 20200104
契約日max 20241231


,proportion
性別,
男性,0.572
女性,0.428


,proportion
契約プラン,
ベーシック,0.486
ライト,0.280
プレミア,0.234


### データのハッシュ化
#### 会員番号をhashlibモジュールを用いて、SHA-256という技術で匿名化加工を行う

In [12]:
def hash_col(col):
    """
    指定したカラムに対して、SHA-256を用いてハッシュ化する。

    Args:
    col(str): ハッシュ化する会員番号（文字列形式）。

    Returns:
    str: 会員番号のハッシュ値（64文字の16進数文字列）。
    """
    hasher = hashlib.sha256()  # SHA-256ハッシュオブジェクトを作成
    hasher.update(col.encode())  # バイト形式にエンコードした項目を追加
    return hasher.hexdigest()  # ハッシュ値を16進数形式の文字列として取得

In [ ]:
# 会員番号をハッシュ化
df_members['会員番号'] = df_members['会員番号'].apply(hash_col)

# 全データのハッシュ化が正しく行われているか確認
assert all(df_members['会員番号'].apply(lambda x: len(x) == 64)), "うまくハッシュ化できていません。"

# ハッシュ化された会員番号がユニークであることを確認
assert df_members['会員番号'].nunique() == 500, "会員番号500件がユニークとなっていません。"

In [14]:
df_members['会員番号'].head()

,会員番号
0,8745e62450a7f27c3944df3045fd2a2ab147356cbc38aa...
1,024ae2da7d9d28b65fcaf35425a52bc44582bc553f93da...
2,bc837ec303f835807dadbf6a141cf3af03151560f82fa3...
3,fd0ac5d67f13eb233bb42d301e0fd8656bf362f66a5e8a...
4,cc45b9ae1f0033849c734cba6e668e472abbfb06304c21...


In [ ]:
# 複数のカラムをハッシュ化させる場合
def hash_columns(df, columns):
    """
    DataFrame内の指定されたカラムに対して、SHA-256を用いてハッシュ化。
    カラムは単一の文字列または文字列のリストとして指定可能。

    Args:
    df (pandas.DataFrame): データフレーム。
    columns (str or list): ハッシュ化するカラム名、またはカラム名のリスト。

    Returns:
    pandas.DataFrame: カラムがハッシュ化されたデータフレーム。
    """
    if isinstance(columns, str):
        # カラム名が単一の文字列であればリストに変換
        columns = [columns]

    def hash_member_id(value):
        hasher = hashlib.sha256()
        hasher.update(str(value).encode())
        return hasher.hexdigest()

    for col in columns:
        # 各カラムに対してハッシュ化を適用
        df[col] = df[col].apply(hash_member_id)
        assert all(df[col].str.len() == 64), f"{col}がうまくハッシュ化できていません。"

    return df


# df = df_members.copy()
# 単一のカラム名を文字列として渡す場合
# df = hash_columns(df, '会員番号')
# 複数のカラム名をリストとして渡す場合（例：会員番号と契約日）
# df = hash_columns(df, ['会員番号', '契約日'])


In [15]:
# ハッシュ値が64文字の文字列になっている。
df_members['会員番号'].apply(len).head()

,会員番号
0,64
1,64
2,64
3,64
4,64


In [ ]:
# CSVファイルに出力
output_path = "./Fitness_Members_Hashed.csv"
df_members.to_csv(output_path, index=False)